# Part 2: CombinedParser Constructors

In [4]:
using CombinedParsers                   ## JuliaCon 2021


using TextParse

### AnyChar

In [2]:
@doc AnyChar

```
AnyChar(T=Char)
```

Parser matching exactly one `x::T`, returning the value.

```jldoctest
julia> AnyChar()
re"."
```


### ValueMatcher

#### CharIn

In [3]:
@doc CharIn

```
CharIn(x)
```

Parser matching exactly one element `c` (character) in a sequence, iif [`_ismatch`](@ref)`(c,x)`.

```jldoctest
julia> a_z = CharIn('a':'z')
re"[a-z]"

julia> parse(a_z, "a")
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

julia> ac = CharIn("ac")
re"[ac]"

julia> parse(ac, "c")
'c': ASCII/Unicode U+0063 (category Ll: Letter, lowercase)

julia> l = CharIn(islowercase)
[islowercase(.)] CharIn
::Char

julia> parse(l, "c")
'c': ASCII/Unicode U+0063 (category Ll: Letter, lowercase)

```

```
CharIn(unicode_category::Symbol...)
```

succeeds if char at cursor is in one of the unicode classes.

```jldoctest
julia> match(CharIn(:L), "aB")
ParseMatch("a")

julia> match(CharIn(:Lu), "aB")
ParseMatch("B")

julia> match(CharIn(:N), "aA1")
ParseMatch("1")
```

Respects boolean logic:

```jldoctest
julia> parse(CharIn(CharIn("ab")),     "a")
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

julia> parse(CharIn(CharNotIn("bc")),  "a")
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

julia> parse(CharNotIn(CharIn("bc")),  "a")
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
```

See also [`CombinedParsers.unicode_classes`](@ref).


#### CharNotIn

In [4]:
@doc CharNotIn

```
CharNotIn(x)
```

Parser matching exactly one element (character) in a sequence, iif not in `x`.

```jldoctest
julia> a_z = CharNotIn('a':'z')
re"[^a-z]"

julia> ac = CharNotIn("ac")
re"[^ac]"

```

Respects boolean logic:

```jldoctest
julia> p = CharNotIn(CharNotIn("ab"));

julia> parse(p,"a")
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
```

```
CharIn(unicode_classes::Symbol...)
```

succeeds if char at cursor is not in any of the `unicode_classes`.


#### Unicode classes

In [5]:
@doc CombinedParsers.unicode_classes

Supported Unicode classes

```jldoctest
julia> for (k,v) in CombinedParsers.unicode_class
         println(":",k, " is a ",v[1],", ", v[2],".")
       end
:L is a Letter, any kind of letter from any language.
:Ll is a Lowercase Letter, a lowercase letter that has an uppercase variant.
:Lu is a Uppercase Letter, an uppercase letter that has a lowercase variant.
:Lt is a Titlecase Letter, a letter that appears at the start of a word when only the first letter of the word is capitalized.
:L& is a Cased Letter, a letter that exists in lowercase and uppercase variants (combination of Ll, Lu and Lt).
:Lm is a Modifier Letter, a special character that is used like a letter.
:Lo is a Other Letter, a letter or ideograph that does not have lowercase and uppercase variants.
:M is a Mark, a character intended to be combined with another character (e.g. accents, umlauts, enclosing boxes, etc.).
:Mn is a Non Spacing Mark, a character intended to be combined with another character without taking up extra space (e.g. accents, umlauts, etc.).
:Mc is a Spacing Combining Mark, a character intended to be combined with another character that takes up extra space (vowel signs in many Eastern languages).
:Me is a Enclosing Mark, a character that encloses the character it is combined with (circle, square, keycap, etc.).
:Z is a Separator, any kind of whitespace or invisible separator.
:Zs is a Space Separator, a whitespace character that is invisible, but does take up space.
:Zl is a Line Separator, line separator character U+2028.
:Zp is a Paragraph Separator, paragraph separator character U+2029.
:S is a Symbol, math symbols, currency signs, dingbats, box-drawing characters, etc..
:Sm is a Math Symbol, any mathematical symbol.
:Sc is a Currency Symbol, any currency sign.
:Sk is a Modifier Symbol, a combining character (mark) as a full character on its own.
:So is a Other Symbol, various symbols that are not math symbols, currency signs, or combining characters.
:N is a Number, any kind of numeric character in any script.
:Nd is a Decimal Digit Number, a digit zero through nine in any script except ideographic scripts.
:Nl is a Letter Number, a number that looks like a letter, such as a Roman numeral.
:No is a Other Number, a superscript or subscript digit, or a number that is not a digit 0–9 (excluding numbers from ideographic scripts).
:P is a Punctuation, any kind of punctuation character.
:Pc is a Connector Punctuation, a punctuation character such as an underscore that connects words.
:Pd is a Dash Punctuation, any kind of hyphen or dash.
:Ps is a Open Punctuation, any kind of opening bracket.
:Pe is a Close Punctuation, any kind of closing bracket.
:Pi is a Initial Punctuation, any kind of opening quote.
:Pf is a Final Punctuation, any kind of closing quote.
:Po is a Other Punctuation, any kind of punctuation character that is not a dash, bracket, quote or connector.
:C is a Other, invisible control characters and unused code points.
:Cc is a Control, an ASCII or Latin-1 control character: 0x00–0x1F and 0x7F–0x9F.
:Cf is a Format, invisible formatting indicator.
:Cs is a Surrogate, one half of a surrogate pair in UTF-16 encoding.
:Co is a Private Use, any code point reserved for private use.
:Cn is a Unassigned, any code point to which no character has been assigned.
```


#### ValueMatcher internal

In [6]:
@doc CombinedParsers.ismatch

```
ismatch(c::MatchingNever,p)
```

returns `false`.

```
ismatch(c,p)
```

returns [`_ismatch`](@ref)`(c, p)`


calls

In [7]:
@doc CombinedParsers._ismatch

```
_ismatch(x::Char, set::Union{Tuple,Vector})::Bool
```

Return `_ismatch(x,set...)`.

```
_ismatch(x, f, r1, r...)
```

Check if `x` matches any of the options `f, r1,r...`: If `ismatch(x,f)` return `true`, otherwise return `_ismatch(x, r1, r...)`.

```
_ismatch(x)
```

returns `false` (out of options)

```
_ismatch(x, p)
```

returns `x==p`

```
_ismatch(c,p::Function)
```

returns `p(c)`

```
_ismatch(c,p::AnyChar)
```

`true`

```
_ismatch(c,p::Union{StepRange,Set})
```

returns `c in p`


### Repeat

In [8]:
@doc Repeat

```
Repeat(x)
Repeat(x; min=0,max=Repeat_max)
Repeat(min::Integer, x)
Repeat(min::Integer,max::Integer, x)
```

Parser repeating pattern `x` `min:max` times.

```jldoctest
julia> Repeat(2,2,'a')
a{2}  |> Repeat
::Array{Char,1}


julia> Repeat(3,'a')
a{3,}  |> Repeat
::Array{Char,1}

```

```
Repeat(f::Function,a...)
```

Abbreviation for `map(f,Repeat(a...))`.


#### Optional

In [9]:
@doc Optional

```
Optional(parser;default=defaultvalue(result_type(parser)))
```

Parser that always succeeds.  If parser succeeds, return result of `parser` with curser behind match. If parser does not succeed, return `default` with curser unchanged.

```jldoctest
julia> match(r"a?","b")
RegexMatch("")

julia> parse(Optional("a", default=42),"b")
42
```


#### Lazy

In [10]:
@doc Lazy

```
Lazy(x::Repeat)
Lazy(x::Optional)
```

Lazy `x` repetition matching (instead of default greedy).

```jldoctest
julia> german_street_address = !Lazy(Repeat(AnyChar())) * Repeat1(' ') * TextParse.Numeric(Int)
🗄 Sequence
├─ .* AnyChar |> Repeat |> !
├─ \  
└─ Int64
Tuple{SubString,Char,Int64}

julia> german_street_address"Konrad Adenauer Allee    42"
("Some Avenue", ' ', 42)
```

!!! note
    PCRE `@re_str`

    ```jldoctest
    julia> re"a+?"
    a+?  |> Repeat |> Lazy
    ::Array{Char,1}

    julia> re"a??"
    a?? |missing |> Lazy
    ::Union{Missing, Char}
    ```



#### Repeat stop
using `NegativeLookahead`

In [11]:
@doc Repeat_stop

```
Repeat_stop(p,stop)
Repeat_stop(p,stop; min=0, max=Repeat_max)
```

Repeat `p` until `stop` (`NegativeLookahead`), not matching `stop`. Sets cursor **before** `stop`. Tries `min:max` times Returns results of `p`.

```jldoctest
julia> p = Repeat_stop(AnyChar(),'b') * AnyChar()
🗄 Sequence
├─ 🗄* Sequence[2] |> Repeat
│  ├─ (?!🗄) NegativeLookahead
│  │  └─ b
│  └─ . AnyChar
└─ . AnyChar
::Tuple{Array{Char,1},Char}

julia> parse(p,"acbX")
(['a', 'c'], 'b')
```

See also [`NegativeLookahead`](@ref)


#### Repeat until

In [12]:
@doc Repeat_until

```
Repeat_until(p,until, with_until=false; wrap=identity, min=0, max=Repeat_max)
```

Repeat `p` until `stop` (with [`Repeat_stop`](@ref)). and set point **after** `stop`.

Return a `Vector{result_type(p)}` if `wrap_until==false`, otherwise a `Tuple{Vector{result_type(p)},result_type(until)}`.

To transform the `Repeat_stop(p)` parser head, provide a function(::Vector{result_type(p)}) in `wrap` keyword argument, e.g.

```jldoctest
julia> p = Repeat_until(AnyChar(),'b') * AnyChar()
🗄 Sequence
├─ 🗄 Sequence[1]
│  ├─ (?>🗄*) Sequence[2] |> Repeat |> Atomic
│  │  ├─ (?!🗄) NegativeLookahead
│  │  │  └─ b
│  │  └─ . AnyChar
│  └─ b
└─ . AnyChar
::Tuple{Array{Char,1},Char}

julia> parse(p,"acbX")
(['a', 'c'], 'X')

julia> parse(Repeat_until(AnyChar(),'b';wrap=JoinSubstring),"acbX")
"ac"
```

See also [`NegativeLookahead`](@ref)


#### join
Oriented at `Base.join` (keyword arg `last` not supported yet)

In [13]:
@doc Base.join

```
join([io::IO,] strings [, delim [, last]])
```

Join an array of `strings` into a single string, inserting the given delimiter (if any) between adjacent strings. If `last` is given, it will be used instead of `delim` between the last two strings. If `io` is given, the result is written to `io` rather than returned as as a `String`.

`strings` can be any iterable over elements `x` which are convertible to strings via `print(io::IOBuffer, x)`. `strings` will be printed to `io`.

# Examples

```jldoctest
julia> join(["apples", "bananas", "pineapples"], ", ", " and ")
"apples, bananas and pineapples"

julia> join([1,2,3,4,5])
"12345"
```

```
Base.join(x::Repeat,delim, infix=:skip)
```

Parser matching repeated `x.parser` separated by `delim`.

```jldoctest
julia> parse(join(Repeat(AnyChar()),','),"a,b,c")
3-element Array{Char,1}:
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 'b': ASCII/Unicode U+0062 (category Ll: Letter, lowercase)
 'c': ASCII/Unicode U+0063 (category Ll: Letter, lowercase)
```

```jldoctest
julia> parse(join(Repeat(AnyChar()),',';infix=:prefix),"a,b,c")
('a', [(',', 'b'), (',', 'c')])

julia> parse(join(Repeat(AnyChar()),',';infix=:suffix),"a,b,c")
([('a', ','), ('b', ',')], 'c')
```

```
Base.join(x::CombinedParser,delim; kw...)
```

Shorthand for `join(Repeat(x),delim; kw...)`.

```
Base.join(f::Function, x::CombinedParser, delim; kw...)
```

Shorthand for `map(f,join(x,delim; kw...))`.


### Atomic

In [14]:
@doc Atomic

```
Atomic(x)
```

A parser matching `p`, and failing when required to backtrack (behaving like an atomic group in regular expressions).


is useful to suppress iterating matches

In [15]:
parse_all(Sequence(Repeat(AnyChar()), Repeat(AnyChar())),"ab") |> collect

6-element Array{Tuple{Array{Char,1},Array{Char,1}},1}:
 (['a', 'b'], [])
 (['a'], ['b'])
 (['a'], [])
 ([], ['a', 'b'])
 ([], ['a'])
 ([], [])

in this case forcing first `Repeat` to be greedy

In [16]:
parse_all(Sequence(Atomic(Repeat(AnyChar())), Repeat(AnyChar())),"ab") |> collect

1-element Array{Tuple{Array{Char,1},Array{Char,1}},1}:
 (['a', 'b'], [])

### Sequence

In [17]:
@doc Sequence

```
Sequence{P,S,T}
```

of `parts::P`, [`sequence_state_type`](@ref)`==S` with [`sequence_result_type`](@ref)`==T`.

```
Sequence(parts::CombinedParser...; tuplestate=false)
```

of `parts`, [`sequence_state_type`](@ref)`(p; tuplestate=tuplestate)` with [`sequence_result_type`](@ref).

Sequences can alternatively created with [`*`](@ref)

```jldoctest
julia> german_street_address = !Repeat(AnyChar()) * ' ' * TextParse.Numeric(Int)
🗄 Sequence
├─ .* AnyChar |> Repeat |> !
├─ \  
└─ Int64
Tuple{SubString,Char,Int64}

julia> german_street_address"Some Avenue 42"
("Some Avenue", ' ', 42)
```

Indexing (transformation) can be defined with

```jldoctest
julia> @syntax german_street_in_address = Sequence(!Repeat(AnyChar()), ' ',TextParse.Numeric(Int))[1]
🗄 Sequence
├─ .* AnyChar |> Repeat |> !
├─ \  
└─ Int64
SubString

julia> german_street_in_address"Some Avenue 42"
"Some Avenue"
```

!!! note
    State is managed as [`sequence_state_type`](@ref)`(parts; tuplestate)`. Overwrite to optimize state types special cases.


```
Sequence(parts...; kw...)
```

Parts that are not `::CombinedParser` are converted with [`parser`](@ref).

```jldoctest
julia> german_street_address = Sequence(!Repeat(AnyChar()), ' ', TextParse.Numeric(Int))
🗄 Sequence
├─ .* AnyChar |> Repeat |> !
├─ \  
└─ Int64
Tuple{SubString,Char,Int64}

julia> german_street_address"Some Avenue 42"
("Some Avenue", ' ', 42)
```

!!! note
    Returns a NamedTuple [`Transformation`](@ref) if any part was `Pair{Symbol}`.

    ```jldoctest
    julia> german_street_address =  Sequence(:street => !Repeat(AnyChar()), " ", :no => TextParse.Numeric(Int))
    🗄 Sequence |> map(ntuple) |> with_name(:german_street_address)
    ├─ .* AnyChar |> Repeat |> ! |> with_name(:street)
    ├─ \  
    └─ Int64  |> with_name(:no)
    ::NamedTuple{(:street, :no),Tuple{SubString,Int64}}

    julia> german_street_address"Some Avenue 42"
    NamedTuple{(:street, :no),Tuple{SubString,Int64}}(("Some Avenue", 42))
    ```


```
Sequence(;kw...)
```

Sequence keyword argument constructors transform the parsing into a named tuple.


Simplifying sequences is used for flattening `*` operator calls
(will probably be refactored to keyword in `Sequence`)

In [18]:
@doc sSequence

```
sSequence(x...)
```

Simplifying `Sequence`, flatten `Sequence`s, remove `Always` assertions.

```jldoctest
julia> Sequence('a',CharIn("AB")*'b')
🗄 Sequence
├─ a
└─ 🗄 Sequence
   ├─ [AB] CharIn
   └─ b
::Tuple{Char,Tuple{Char,Char}}


julia> sSequence('a',CharIn("AB")*'b')
🗄 Sequence
├─ a
├─ [AB] CharIn
└─ b
::Tuple{Char,Char,Char}
```

See also [`Sequence`](@ref)


Flexible `Sequence` state:

In [19]:
@doc CombinedParsers.sequence_state_type

```
sequence_state_type(pts::Type; tuplestate=false)
```

  * `MatchState` if all `fieldtypes` are `MatchState`,
  * otherwise if `tuplestate`, a tuple type with the `state_type` of `parts`,
  * or `Vector{Any}` if `!tuplestate`.

!!! note
    Todo: NCodeunitsState instead of MatchState might increase performance.



`Sequence` results in a `Tuple` by default:

In [20]:
@doc CombinedParsers.sequence_result_type

```
sequence_result_type(::Type{T}) where {T<:Tuple}
```

`Tuple` type, internally used for `Sequence` result_type.


### Either
More later for recursive parsing

In [21]:
@doc Either

Parser that tries matching the provided parsers in order, accepting the first match, and fails if all parsers fail.

This parser has no `==` and `hash` methods because it can recurse.

```jldoctest
julia> match(r"a|bc","bc")
RegexMatch("bc")

julia> parse(Either("a","bc"),"bc")
"bc"

julia> parse("a" | "bc","bc")
"bc"

```

```
Either(p...)
```

Create a immutable `Either{<:Tuple}` improved for performance. Arguments `p...` are wrapped in [`parser`](@ref), type parameters are computed with [`either_state_type`](@ref) and [`either_result_type`](@ref).

```
Either(p::Vector)
```

Create a mutable `Either{Vector{Any}}` for creating recursive parsers. Arguments `p...` are wrapped in [`parser`](@ref), type parameters are computed with [`either_state_type`](@ref) and [`either_result_type`](@ref).

See also [`@syntax`](@ref).

!!! note
    state type and result type are `Any` which might cost performance.


```
Either{T}(p...)
```

Create a mutable `Either{<:Vector{Any}}` for creating recursive parsers. Options can be added with [`push!`](@ref) and [`pushfirst!`](@ref).

See also [`@syntax`](@ref).

!!! note
    state type is `Any` which might cost performance.


```
Either(transform::Function, x::Vararg)
```

abbreviation for `map(transform, Either(x...))`.


### Assertions

In [22]:
@doc AtStart

```
AtStart()
```

Parser succeding if and only if at index 1 with `result_type` `AtStart`.

```jldoctest
julia> AtStart()
re"^"

```


In [23]:
@doc AtEnd

```
AtEnd()
```

Parser succeding if and only if at last index with `result_type` `AtEnd`.

```jldoctest
julia> AtEnd()
re"$"

```


In [24]:
@doc Always

```
Always()
```

Assertion parser matching always and not consuming any input. Returns `Always()`.

```jldoctest
julia> Always()
re""

```


In [25]:
@doc Never

```
Never()
```

Assertion parser matching never.

```jldoctest
julia> Never()
re"(*FAIL)"

```


#### Look behind

In [26]:
@doc PositiveLookbehind

```
PositiveLookbehind(parser)
```

Parser that succeeds if and only if `parser` succeeds **before cursor**. Consumes no input. The match is returned. Useful for checks like "must be preceded by `parser`, don't consume its match".


In [27]:
@doc NegativeLookbehind

```
NegativeLookbehind(parser)
```

Parser that succeeds if and only if `parser` does not succeed **before cursor**.  Consumes no input. `nothing` is returned as match. Useful for checks like "must not be preceded by `parser`, don't consume its match".

```jldoctest
julia> la=NegativeLookbehind("keep")
re"(?<!keep)"

julia> parse("peek"*la,"peek")
("peek", re"(?<!keep)")
```


#### Look ahead

In [28]:
@doc PositiveLookahead

```
PositiveLookahead(parser)
```

Parser that succeeds if and only if `parser` succeeds, but consumes no input. The match is returned. Useful for checks like "must be followed by `parser`, but don't consume its match".

```jldoctest
julia> la=PositiveLookahead("peek")
re"(?=peek)"

julia> parse(la*AnyChar(),"peek")
("peek", 'p')

```


In [29]:
@doc NegativeLookahead

```
NegativeLookahead(parser)
```

Parser that succeeds if and only if `parser` does not succeed, but consumes no input. `parser` is returned as match. Useful for checks like "must not be followed by `parser`, don't consume its match".

```jldoctest
julia> la = NegativeLookahead("peek")
re"(?!peek)"

julia> parse(la*AnyChar(),"seek")
(re"(?!peek)", 's')

```


### Logging and Side-Effects

In [30]:
@doc with_name

```
with_name(name::Symbol,x; doc="")
```

A parser labelled with `name`. Labels are useful in printing and logging.

See also: [`@with_names`](@ref), [`with_name`](@ref), [`log_names`](@ref)


In [31]:
@doc @with_names

```
@with_names
```

Sets names of parsers within begin/end block to match the variables they are asigned to.

so, for example

```jldoctest
julia> @with_names foo = AnyChar()
. AnyChar |> with_name(:foo)
::Char

julia> parse(log_names(foo),"ab")
   match foo@1-2: ab
                  ^
'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
```

See also [`log_names(parser)`](@ref), [`@syntax`](@ref).


In [32]:
@doc log_names

```
log_names(x,names=true; exclude=nothing)
```

Rebuild parser replacing `NamedParser` instances with `with_log` parsers. Log all `NamedParser` instanses if `names==true` or `name in names` and not `name in exclude`.

See also: [`with_log`](@ref), [`deepmap_parser`](@ref)


In [33]:
@doc with_log

```
with_log(s::AbstractString,p, delta=5;nomatch=false)
```

Log matching process of parser `p`, displaying `delta` characters left of and right of match.

If `nomatch==true`, also log when parser does not match.

See also: [`log_names`](@ref), [`with_effect`](@ref)


In [34]:
@doc with_effect

```
with_effect(f::Function,p,a...)
```

Call `f(sequence,before_i,after_i,state,a...)` if `p` matches,  `f(sequence,before_i,before_i,nothing,a...)` otherwise.


### Caching match states

In [35]:
@doc CombinedParsers.WithMemory

```
WithMemory(x) <: AbstractString
```

String wrapper with memoization of next match states for parsers at indices. Memoization is sometimes recommended as a way of improving the performance of parser combinators (like state machine optimization and compilation for regular languages).

!!! note
    A snappy performance gain could not be demonstrated so far, probably because the costs of state memory allocation for caching are often greater than recomputing a match.  If you have a case where your performance benefits with this, let me know!


```


## Recursive Parsers

# Arithmetical terms for rational numbers
Parsing is reading and transforming a sequence of characters.
This example reads and evaluates arithmetical terms for rational numbers.
Subterms can use algebraic operators `+-*/` that will be evaluated with

In [1]:
function evaluate( (start, operation_values) )
    aggregated_value::Rational{Int} = start
    for (op,val) in operation_values
        aggregated_value = eval( Expr(:call, Symbol(op),
			              aggregated_value, val
			              ))
    end
    return aggregated_value
end
evaluate( (0, [ ('+',1), ('+',1) ]) ),
evaluate( (1, [ ('*',2), ('*',3) ]) )

(2//1, 6//1)

Term expressions are sequences of subterms interleaved with operators.

Sub terms can be several things, and recursively nested. `Either` is used for recursion. 

In [37]:
subterm = Either{Rational{Int}}(Any[]);

A term can be an addition/subtraction of a multiplication/division of subterms.

A subterm is either an integer or a nested term in parentheses.

In [6]:
mult = map(evaluate, join(subterm, CharIn("*/"), infix=:prefix ))
term = map(evaluate, join(mult,    CharIn("+-"), infix=:prefix ))
parenthesis = Sequence(2,'(',term,')')

push!(subterm, parenthesis)
push!(subterm, TextParse.Numeric(Int))


term("(1+2)/5")

3//5

Term expressions are sequences of subterms interleaved with operators.
Sub terms are `Either` fast `TextParse.Numeric(Int)` integer numbers, converted to `Rational{Int}`,

In [8]:
@syntax subterm = Either{Rational{Int}}(
        Any[map(Rational{Int},
                parser(TextParse.Numeric(Int)))])

|🗄 Either |> with_name(:subterm)
└─  <Int64> |> map(Rational{Int64})
::Rational{Int64}


A subterm can also be a nested term in parentheses

In [9]:
@syntax for parenthesis in subterm
    mult = evaluate |> join(subterm, CharIn("*/"), infix=:prefix )
    adds = evaluate |> join(mult,    CharIn("+-"), infix=:prefix )
    Sequence(2,'(',adds,')')
end;
subterm

|🗄 Either |> with_name(:subterm)
├─ 🗄 Sequence |> map(#55) |> with_name(:parenthesis)
│  ├─ \( 
│  ├─ 🗄 Sequence |> map(evaluate)
│  │  ├─ 🗄 Sequence |> map(evaluate)
│  │  │  ├─ |🗄 Either |> with_name(:subterm) # branches hidden
│  │  │  └─ 🗄* Sequence |> Repeat
│  │  │     ├─ [\*/] CharIn
│  │  │     └─ |🗄 Either |> with_name(:subterm) # branches hidden
│  │  └─ 🗄* Sequence |> Repeat
│  │     ├─ [\+\-] CharIn
│  │     └─ 🗄 Sequence |> map(evaluate) # branches hidden
│  └─ \) 
└─  <Int64> |> map(Rational{Int64})
::Rational{Int64}


This `CombinedParser` definition in a few lines is sufficient for doing arithmetics:
`Base.join(x,infix; infix=:prefix)` is shorthand for `x * Repeat( infix * x  )`,
and `f |> parser` is shorthand for `map(f,parser)`.

In [11]:
@syntax term = adds;

registers a `@term_string` macro for parsing and transforming.

In [42]:
term"(1+2)/5"

3//5

The defined `CombinedParser` `term` can be used as a function for colorful logging of the parsing process.

In [12]:
term("1/((1+2)*4+3*(5*2))",log = [:parenthesis])

   match parenthesis@4-9: 1/((1+2)*4+3*(
   match parenthesis@14-19: *4+3*(5*2))
   match parenthesis@3-20: 1/((1+2)*4+3*(5*2))


1//42

[Is every rational answer ultimately the inverse of a universal question in life?](https://en.wikipedia.org/wiki/Phrases_from_The_Hitchhiker%27s_Guide_to_the_Galaxy#Answer_to_the_Ultimate_Question_of_Life,_the_Universe,_and_Everything_(42))

The parser representation can be printed as a tree

In [13]:
term

🗄 Sequence |> map(evaluate) |> with_name(:term)
├─ 🗄 Sequence |> map(evaluate)
│  ├─ |🗄 Either |> with_name(:subterm)
│  │  ├─ 🗄 Sequence |> map(#55) |> with_name(:parenthesis)
│  │  │  ├─ \( 
│  │  │  ├─ 🗄 Sequence |> map(evaluate)
│  │  │  │  ├─ 🗄 Sequence |> map(evaluate) # branches hidden
│  │  │  │  └─ 🗄* Sequence |> Repeat # branches hidden
│  │  │  └─ \) 
│  │  └─  <Int64> |> map(Rational{Int64})
│  └─ 🗄* Sequence |> Repeat
│     ├─ [\*/] CharIn
│     └─ |🗄 Either |> with_name(:subterm) # branches hidden
└─ 🗄* Sequence |> Repeat
   ├─ [\+\-] CharIn
   └─ 🗄 Sequence |> map(evaluate) # branches hidden
::Rational{Int64}


### Benchmarks
Parsing times for Int, operators, brackets are

In [45]:
using BenchmarkTools
@benchmark match(term,"(1+2)/5")

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.228 μs …  1.224 ms  ┊ GC (min … max):  0.00% … 99.64%
 Time  (median):     2.377 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.083 μs ± 26.653 μs  ┊ GC (mean ± σ):  19.30% ±  2.23%

   ▂█▆▂                                                       
  ▂████▆▄▄▄▄▄▅▄▃▂▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.23 μs        Histogram: frequency by time         4.1 μs <

 Memory estimate: 2.50 KiB, allocs estimate: 47.

in unfair benchmark-comparison with the more expressive Julia syntax parser

In [46]:
@benchmark Meta.parse("(1+2)/5")

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  57.041 μs …   4.472 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     58.611 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.708 μs ± 191.040 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃██▅▁    ▁▂        ▁                                         ▁
  █████▇█████▇███▇▇▇███▇▆▆▅▆▄▅▆▄▅▅▄▅▄▅▅▄▃▄▄▄▃▃▃▄▆▅▆▇█▇▅▄▃▂▂▂▄▃ █
  57 μs         Histogram: log(frequency) by time      98.4 μs <

 Memory estimate: 624 bytes, allocs estimate: 14.

Parsing and transforming (here `eval`)

In [47]:
@benchmark term("(1+2)/5")

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  176.684 μs …  11.133 ms  ┊ GC (min … max): 0.00% … 97.30%
 Time  (median):     180.931 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   218.908 μs ± 381.599 μs  ┊ GC (mean ± σ):  1.44% ±  1.69%

  ▃█▇▅▄▃▃▂▁▁▁▁▁▁  ▁  ▁                                          ▂
  ████████████████████████▇▇▇▇▇▆▅▄▅▅▆▅▄▅▅▅▅▅▄▄▄▅▅▄▆▅▅▅▄▅▆▅▄▁▃▄▅ █
  177 μs        Histogram: log(frequency) by time        301 μs <

 Memory estimate: 14.59 KiB, allocs estimate: 262.

compared to Julia

In [48]:
@benchmark eval(Meta.parse("(1+2)/5"))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  140.329 μs …   7.099 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     167.133 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   213.659 μs ± 373.456 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  █▄▂▂▂▅▅█▅▅▄▃▂▂▂▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  140 μs           Histogram: frequency by time          315 μs <

 Memory estimate: 3.14 KiB, allocs estimate: 52.

recursive parsers can be built with Either

In [49]:
@doc push!

```
push!(collection, items...) -> collection
```

Insert one or more `items` in `collection`. If `collection` is an ordered container, the items are inserted at the end (in the given order).

# Examples

```jldoctest
julia> push!([1, 2, 3], 4, 5, 6)
6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
```

If `collection` is ordered, use [`append!`](@ref) to add all the elements of another collection to it. The result of the preceding example is equivalent to `append!([1, 2, 3], [4, 5, 6])`. For `AbstractSet` objects, [`union!`](@ref) can be used instead.

```
push!(q::Deque{T}, x)
```

Add an element to the back

```
push!(s::IntDisjointSets{T})
```

Make a new subset with an automatically chosen new element x. Returns the new element. Throw an `ArgumentError` if the capacity of the set would be exceeded.

```
push!(s::DisjointSets{T}, x::T)
```

Make a new subset with an automatically chosen new element x. Returns the new element.

```
push!(h::BinaryHeap, value)
```

Adds the `value` element to the heap `h`.

```
push!(sc, k=>v)
```

Argument `sc` is a SortedDict or SortedMultiDict and `k=>v` is a key-value pair. This inserts the key-value pair into the container. If the key is already present, this overwrites the old value. The return value is `sc`. Time: O(*c* log *n*)

```
push!(sc, k=>v)
```

Argument `sc` is a SortedDict or SortedMultiDict and `k=>v` is a key-value pair. This inserts the key-value pair into the container. If the key is already present, this overwrites the old value. The return value is `sc`. Time: O(*c* log *n*)

```
push!(sc, k)
```

Argument `sc` is a SortedSet and `k` is a key. This inserts the key into the container. If the key is already present, this overwrites the old value. (This is not necessarily a no-op; see below for remarks about the customizing the sort order.) The return value is `sc`. Time: O(*c* log *n*)

```
push!(cb::CircularBuffer, data)
```

Add an element to the back and overwrite front if full.

```
push!(tree, key)
```

Inserts `key` in the `tree` if it is not present.

```
Base.push!(x::Either, option)
```

Push `option` to `x.options` as parser tried next if `x` fails.

Recursive parsers can be built with `push!` to `Either`.

See also [`pushfirst!`](@ref) and [`@syntax`](@ref).

```
Base.push!(x::WrappedParser{<:Either}, option)
```

Push `option` to `x.options` of repeated inner parser.


new options can also take precendence

In [50]:
@doc pushfirst!

```
pushfirst!(collection, items...) -> collection
```

Insert one or more `items` at the beginning of `collection`.

# Examples

```jldoctest
julia> pushfirst!([1, 2, 3, 4], 5, 6)
6-element Array{Int64,1}:
 5
 6
 1
 2
 3
 4
```

```
pushfirst!(q::Deque{T}, x)
```

Add an element to the front

```
pushfirst!(D::CircularDeque, v)
```

Add an element to the front.

```
pushfirst!(cb::CircularBuffer, data)
```

Insert one or more items at the beginning of CircularBuffer and overwrite back if full.

```
Base.pushfirst!(x::Either, option)
```

Push `option` to `x.options` as parser tried first, and trying `x` if `option` fails.

Recursive parsers can be built with `pushfirst!` to `Either`.

See also [`push!`](@ref) and [`@syntax`](@ref).

```
Base.pushfirst!(x::WrappedParser{<:Either}, option)
```

Push `option` as first `x.options` of repeated inner parser.


The `@syntax for option in either` can also be used

In [51]:
@doc @syntax

```
@syntax name = expr
```

Convenience macro defining a CombinedParser `name=expr` and custom parsing macro `@name_str`.

```jldoctest
julia> @syntax a = AnyChar()

julia> a"char"

```

```
@syntax for name in either; expr; end
```

Parser `expr` is [`pushfirst!`](@ref) to `either`. If `either` is undefined, it will be created. If `either == :text || either == Symbol(:)` the parser will be added to `CombinedParser_globals` variable in your module.

```jldoctest
julia> @syntax street_address = Either(Any[]);

julia> @syntax for german_street_address in street_address
            Sequence(!!Repeat(AnyChar()),
                     " ",
                     TextParse.Numeric(Int)) do v
                (street = v[1], no=v[3])
            end
       end
🗄 Sequence |> map(#50) |> with_name(:german_street_address)
├─ .* AnyChar |> Repeat |> ! |> map(intern) |> map(String)
├─ \  
└─  <Int64>
::NamedTuple{(:street, :no),Tuple{String,Int64}}

julia> german_street_address"Some Avenue 42"
NamedTuple{(:street, :no),Tuple{SubString,Int64}}(("Some Avenue", 42))


julia> @syntax for us_street_address in street_address
            Sequence(TextParse.Numeric(Int),
                     " ",
                     !!Repeat(AnyChar())) do v
                (street = v[3], no=v[1])
            end
       end
🗄 Sequence |> map(#52) |> with_name(:us_street_address)
├─  <Int64>
├─ \  
└─ .* AnyChar |> Repeat |> ! |> map(intern) |> map(String)
::NamedTuple{(:street, :no),Tuple{String,Int64}}

julia> street_address"50 Oakland Ave"
(street = "Oakland Ave", no = 50)

julia> street_address"Oakland Ave 50"
(street = "Oakland Ave", no = 50)
```


## EBNF support
BNF parser draft:
- left recursion draft not published
If you want to invest time or resources I am very happy to collaborate.

Left-recursion is a pattern in metalanguages, a pattern when defining what texts are valid in languages.
For example in the language of arithmetic, of e.g. addition of numbers:

    1+3+8+2

A language for such expressions is defined with

    expression: expression "+" expression | number
    number: [0-9]+

written in the popular EBNF format for metalanguages.
In this representation expressions can contain expressions and so on.

In this representation expressions can contain expressions and so on.

    1+3
       +8
         +2

This would also be a valid parsing (if precedence is not enforced):

    1+
      3+
        8+2

ParserCombinators resort to a `Either/Delayed` function for recursion,
as does `CombinedParsers.jl`.
Personally I found this an intuitive and complete alternative.

Such left recursion is a problem for creating a Parser Combinator with julia parser package CombinedParsers.jl.
A parser combinator descends into functions that check for a match at a position, in the expression check, for the first decision, such a function descends, and an infinite call stack overflow results.

Most of the time it is possible to prevent left recursion by reformulating the parser.

A left recursive CombinedParsers will throw a StackOverflowError.
Left-recursive parsers can be rewritten nestedly CombinedParsers, similar to inserting log statements, with the `deepmap_parser` heuristic, inserting a Memoization parser that blocks recursion.

However, implementation is not yet complete.
One of `CombinedParsers` strengths is to iterate all matches, if ambigous.


A CombinedParser syntax draft for parsing BNF: <https://github.com/gkappler/CombinedParsers.jl/blob/master/test/bnf.jl> 

Happy if you want to get involved!

# Regular Expressions
the standard to parse (`Base.PCRE` C library).
#### **Construct** `CombinedParsers` from `Regex` string:

In [52]:
speaker = "Gregor Kappler
psychometric scientist and programmer, independent";

using CombinedParsers.Regexp

import CombinedParsers.Regexp: word, words, whitespace # hide

person_background = Sequence(
    :name => !words, whitespace, :lastname => !word, "\n",
    :background => !words)

r = re"(?<name>[\w\h]+)\h+(?<lastname>\w+)\n(?<background>[\w\h]+)"

🗄 Sequence |> regular expression combinator with 3 capturing groups
├─ (?<name>[\ \ \u180e\ \ \ \ \ \ \ \ \ \ \ \　\ \ \t_\ \p{1:5}\p{9:11}]+) CharIn |> Repeat |> Capture 1 |> with_name(:name)
├─ [\h]+ CharIn |> Repeat
├─ (?<lastname>[\w]+) CharIn |> Repeat |> Capture 2 |> with_name(:lastname)
├─ \n 
└─ (?<background>[\ \ \u180e\ \ \ \ \ \ \ \ \ \ \ \　\ \ \t_\ \p{1:5}\p{9:11}]+) CharIn |> Repeat |> Capture 3 |> with_name(:background)
::Tuple{Array{Char,1},Array{Char,1},Array{Char,1},Char,Array{Char,1}}


#### **Understand structure** of `Regex` side-by-side with `CombinedParsers` API.

## PCRE compliance: `match` API
Use `CombinedParsers`
like `Base.RegexMatch`

In [53]:
m = match(r, speaker)
m[:name], m.stop

("Gregor", 53)

Transform strings into any Julia type e.g. `NamedTuple`

In [54]:
parse(person_background, speaker)

(name = "Gregor", lastname = "Kappler", background = "psychometric scientist and programmer")

When captures in `Regex` repeatedly match, only the last match is captured.

With `CombinedParsers` you can
transform  with `map` a String into any Julia representation with nesting and capturing all repeated values.

Julia `result_type` inference does make
parsers brief without redundant type definitions.
In fact, a CombinedParser defines the types of a data domain from the syntax definition.

## PCRE compliance: Features

✅ 3071 on 972 patterns,<br/>
❌ 25 on 17 patterns <br/>
(PCRE C library unit tests).

**not supported**<br/>
❌ `ACCEPT, SKIP, COMMIT, THEN, PRUNE, \K`
<br/>
❌ capture, lookaheads in lookbehinds

**Characters**<br/>
✅ escapes, character types
<br/>
✅ character ranges (`[]`)

**Basics**<br/>
✅ sequences, `|`, `?`
<br/>
✅ (lazy) repetitions
<br/>
✅ atomic groups
<br/>
✅ options

**Groups**<br/>
✅ non-capturing `(?:)`
<br/>
✅ capturing groups `()`
<br/>
✅ backreferences `\1` & subroutines `(?1)`
<br/>
✅ conditional expressions

**Assertions**<br/>
✅ assertions `\z\Z\b\B^$`...
<br/>
✅ look-aheads & -behinds

# Regex ↔ Building Blocks
#### `@re_str::CombinedParser` conforming with `Base.PCRE` [spec](https://www.pcre.org/original/doc/html/pcrepattern.html)
- reading PCRE regular expressions and
- resulting in a `CombinedParser` usable as plug-in `Regex` replacement

# Brevity
Julia's type inference
saves you time and inconsistencies of writing type information repeatedly during data-preparation.

For me parsing was often tedious, involving redundant definitions.
Even fluent, parsing stays some sort pain, yes or no?
`CombinedParsers` feel much more fun at greater output.
`CombinedParsers` requires very little little repeated information,
Julia type inference is so useful!

# Iteration combinatorics
#### lazy `parse_all`

In [55]:
[ e for e in parse_all(re"^(a|ab|b)+$","abab") ]

4-element Array{Tuple{AtStart,Array{Union{Char, Tuple{Char,Char}},1},AtEnd},1}:
 (re"^", ['a', 'b', 'a', 'b'], re"$")
 (re"^", ['a', 'b', ('a', 'b')], re"$")
 (re"^", [('a', 'b'), 'a', 'b'], re"$")
 (re"^", [('a', 'b'), ('a', 'b')], re"$")

If a parsing is not uniquely defined
Julia's `iterate` API lazily `parse`s with backtracking combinatorics,

#### lazy `match_all`

In [56]:
[ (e.start,get(e)) for e in match_all(re"(a|ab|b)+","ab") ]

4-element Array{Tuple{Int64,Array{Union{Char, Tuple{Char,Char}},1}},1}:
 (1, ['a', 'b'])
 (1, ['a'])
 (1, [('a', 'b')])
 (2, ['b'])

## Parametric Types & Dispatch Optimization
Parsers and states have custom types.
This allows method dispatch
to insert fast matching code for special-cases.

This is used in lazy construction of results based on an appropriate (sub-)state.

Supports currently

In [57]:
rp = Repeat(AnyChar())
match(rp,"0.1.2.3").state

7

State is just the chars matched

In [58]:
match(rp,"0.1.2.3")[3]

ParseMatch("1")

and

In [59]:
sp = Sequence(AnyChar(),AnyChar(),AnyChar(),AnyChar())
match(sp,"0.1.2.3").state

∘

State is `MatchState`

In [60]:
match(sp,"0.1.2.3")[3]

ParseMatch("1")

The implementation shows how state is managed in CombinedParsers: <https://github.com/gkappler/CombinedParsers.jl/blob/master/src/lazy.jl>

Another Example: custom palindrome parser API in [docs](https://gkappler.github.io/CombinedParsers.jl/dev/man/example-palindromes/).

Optimization is ongoing.

# Prefix Trees

In [61]:
using Random
using BenchmarkTools
s = [ randstring(5) for _ in 1:5000 ];

Matching one of 5000 `String`s in `s` is slow in PCRE (5 characters each):

In [62]:
re = Regex(join(s,"|"));
median(@benchmark match($re,$s[end]))

BenchmarkTools.TrialEstimate: 
  time:             24.405 μs
  gctime:           0.000 ns (0.00%)
  memory:           224 bytes
  allocs:           3

PCRE fails when matching any of 10000 Strings.
`CombinedParsers` dispatches `Either(::Vector{<:AbstractString})`
to create an `Either{Trie{Char}}` (no state machines)
with a fast `_iterate` implementation, and no limit to the number of strings.

In [63]:
pc = Either(s);
median(@benchmark match($pc,$s[end]))

BenchmarkTools.TrialEstimate: 
  time:             111.052 ns
  gctime:           0.000 ns (0.00%)
  memory:           256 bytes
  allocs:           3

## Summary
#### 1. Regular Expressions
- PCRE-compliance
- building blocks
- inter-operability

#### 2. Design and Standards
- brevity ← type inference
- iteration combinatorics
- custom parser API

#### 3. Performance
- PCRE benchmarks
- parametric types
- dispatch optimization
- prefix trees
- memoization

# CombinedParsers Packages
- [Tries.jl](https://github.com/gkappler/Tries.jl) backend of fast prefix-tree matching (see [docs](https://gkappler.github.io/CombinedParsers.jl/dev/man/example-either-trie/))
- [CombinedParserTools.jl](https://github.com/gkappler/CombinedParserTools.jl) for re-useable parsers, markup and annotation types.
- [WikitextParser.jl](https://github.com/gkappler/WikitextParser.jl) for [wikitext syntax](https://en.wikipedia.org/wiki/Help:Wikitext), with markup types.
- OrgmodeParser.jl for [org mode](https://orgmode.org/) syntax.

If you want to invest in creating a message broker with `CombinedParsers`, I will gladly collaborate.
If you want to work with `CombinedParsers`, I will gladly provide professional support.
If you are writing your own recursive `CombinedParser` and seek inspiration, you might find these comprehensive examples interesting.

# `CombinedParsers` Design
- **fast**: optimizable with Julia method dispatch, generated functions,
- **brief**: type-inference defines the domain data types,
- **inter-operable**: composable with `Regex`, `TextParse`...
- **general**: provides flexible public API for matching, parsing, iteration

### Next
- Syntax freeze (no `Char`)
- Optimization experiments
- error tracing (debugging?)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*